# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [ ]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [ ]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

In [ ]:
customer_products

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [ ]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [ ]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [ ]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

In [ ]:
prod_cust_pivot.head()

In [ ]:
cust_prod_pivot.head()

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [ ]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [63]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,Pasta - Angel Hair,Tomatoes Tear Drop,Grouper - Fresh,Vaccum Bag 10x13
2,264,Scallops - 10/20,Wine - Ej Gallo Sierra Valley,Cheese - Camembert,Corn Meal,Browning Caramel Glace
3,356,Squid U5 - Thailand,Clam Nectar,"Turnip - White, Organic","Yogurt - Blueberry, 175 Gr",Garbag Bags - Black
4,412,Beef - Montreal Smoked Brisket,Scallops - 10/20,Onions - Cippolini,Juice - Orange,Cake - Box Window 10x10x2.5


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [64]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 33759, 6001, 79458]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [65]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [66]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [67]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot.T, 'euclidean'))),
                         index=cust_prod_pivot.columns, columns=cust_prod_pivot.columns)

In [68]:
cust_dist

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.796742,0.791464,0.789384,0.804115,0.792667,0.798399,0.793217,0.803609,0.802807,...,0.803269,0.810978,0.801139,0.786881,0.790177,0.797245,0.793445,0.803000,0.801386,0.797058
"Appetizer - Mini Egg Roll, Shrimp",0.796742,1.000000,0.793782,0.791250,0.805458,0.798576,0.800446,0.801148,0.803247,0.798930,...,0.802403,0.808219,0.803494,0.790071,0.793299,0.788344,0.795851,0.805978,0.804443,0.800032
Appetizer - Mushroom Tart,0.791464,0.793782,1.000000,0.786451,0.794867,0.793002,0.795558,0.790171,0.796368,0.791388,...,0.794889,0.802256,0.798552,0.791591,0.792390,0.785607,0.792052,0.795823,0.798405,0.791327
Appetizer - Sausage Rolls,0.789384,0.791250,0.786451,1.000000,0.792416,0.790813,0.788830,0.785863,0.786814,0.788055,...,0.791351,0.800896,0.795318,0.789134,0.782490,0.776849,0.790387,0.793389,0.792520,0.787578
Apricots - Dried,0.804115,0.805458,0.794867,0.792416,1.000000,0.805097,0.802059,0.800923,0.803156,0.805115,...,0.803241,0.810797,0.807071,0.796874,0.795336,0.789832,0.802296,0.805877,0.809478,0.802340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.797245,0.788344,0.785607,0.776849,0.789832,0.789979,0.792104,0.786584,0.790960,0.794622,...,0.794903,0.795676,0.792507,0.778380,0.784424,1.000000,0.782226,0.790713,0.793388,0.787416
Yoghurt Tubes,0.793445,0.795851,0.792052,0.790387,0.802296,0.793450,0.802114,0.794246,0.796563,0.795867,...,0.798597,0.803393,0.801791,0.784569,0.791333,0.782226,1.000000,0.794140,0.797547,0.795404
"Yogurt - Blueberry, 175 Gr",0.803000,0.805978,0.795823,0.793389,0.805877,0.808906,0.807825,0.800803,0.806360,0.800552,...,0.809772,0.811418,0.805326,0.788959,0.793862,0.790713,0.794140,1.000000,0.807749,0.800157


### Step 2: Get the products purchased for a specific customer of your choice.

In [69]:
purchase_prodcuts = cust_prod_pivot.loc[33][cust_prod_pivot.loc[33] > 0]
purchase_prodcuts

ProductName
Apricots - Dried                     0.015191
Assorted Desserts                    0.014672
Bandage - Flexible Neon              0.013172
Bar Mix - Pina Colada, 355 Ml        0.014820
Beans - Kidney, Canned               0.015327
Beef - Chuck, Boneless               0.014651
Beef - Prime Rib Aaa                 0.017736
Beer - Original Organic Lager        0.014834
Beer - Rickards Red                  0.015296
Black Currants                       0.016540
Blackberries                         0.016479
Bread - Italian Roll With Herbs      0.013630
Bread Foccacia Whole                 0.014637
Cake - Box Window 10x10x2.5          0.013994
Cassis                               0.017705
Cheese - Cottage Cheese              0.015821
Crab - Imitation Flakes              0.015653
Fondant - Icing                      0.016374
French Pastry - Mini Chocolate       0.013262
General Purpose Trigger              0.016576
Grapes - Red                         0.017746
Grouper - Fresh       

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [70]:
items_purchased = [index for index,value in purchase_prodcuts.items()]

len(items_purchased)

59

In [71]:
key_list = []

for item in items_purchased:
    top5 = cust_dist[item].sort_values(ascending=False)[1:].nlargest(5)
    key_list.append([keys for keys,value in top5.items()])

flat_list = [item for sublist in key_list for item in sublist]

len(flat_list)
             
  

295

In [72]:
filter_list = [item for item in flat_list if item not in items_purchased]
len(filter_list)

246

In [73]:
filter_list

['Bread - Roll, Soft White Round',
 'Coffee Decaf Colombian',
 'Onion Powder',
 'Kiwi',
 'Bread - Rye',
 'Puree - Passion Fruit',
 'Flour - Whole Wheat',
 'Snapple - Iced Tea Peach',
 'Kiwi',
 'Veal - Osso Bucco',
 'Cookies - Assorted',
 'Flavouring - Orange',
 'Oil - Shortening - All - Purpose',
 'Beans - Wax',
 'Veal - Osso Bucco',
 'Squid U5 - Thailand',
 'Onion Powder',
 'Garbage Bags - Clear',
 'Wine - Magnotta - Belpaese',
 'Sun - Dried Tomatoes',
 'Pepper - Paprika, Hungarian',
 'Wine - Cahors Ac 2000, Clos',
 'Onion Powder',
 'Bread - Roll, Soft White Round',
 'Wine - Magnotta - Belpaese',
 'Wine - White, Mosel Gold',
 'Cheese - Brie,danish',
 'Bread - Roll, Soft White Round',
 'Pastry - Choclate Baked',
 'Wine - Magnotta - Belpaese',
 'Wine - White Cab Sauv.on',
 'Creme De Banane - Marie',
 'Bread - Roll, Soft White Round',
 'Wine - White, Mosel Gold',
 'Wine - Magnotta - Belpaese',
 'Soup - Canadian Pea, Dry Mix',
 'Wine - Magnotta - Belpaese',
 'Veal - Osso Bucco',
 'Ocean S

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [74]:
from collections import Counter

countera = dict(Counter(filter_list))
countera
answer = [(key,countera[key]) for key in countera.keys()]
answer
answer_sorted = sorted(answer, key=lambda x: x[1],reverse=True)
answer_sorted = [tupl[0] for tupl in answer_sorted]
answer_sorted = answer_sorted[:5]
answer_sorted

['Bread - Roll, Soft White Round',
 'Veal - Osso Bucco',
 'Wine - Magnotta - Belpaese',
 'Onion Powder',
 'Wine - White, Mosel Gold']

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [75]:
recomendations = {}

In [76]:
customers = list(customer_products['CustomerID'].unique())

In [77]:
key_list = []
for cust in customers:
    purchase_prodcuts = cust_prod_pivot.loc[cust][cust_prod_pivot.loc[cust] > 0]
    items_purchased = [index for index,value in purchase_prodcuts.items()]
for item in items_purchased:
    top5 = cust_dist[item].sort_values(ascending=False)[1:].nlargest(5)
    key_list.append([keys for keys,value in top5.items()])

    flat_list = [item for sublist in key_list for item in sublist]
    filter_list = [item for item in flat_list if item not in items_purchased]

    countera = dict(Counter(filter_list))
    countera
    answer = [(key,countera[key]) for key in countera.keys()]
    answer
    answer_sorted = sorted(answer, key=lambda x: x[1],reverse=True)
    answer_sorted = [tupl[0] for tupl in answer_sorted]
    answer_sorted = answer_sorted[:5]
    recomendations[cust] = list(answer_sorted)
recomendations
    

{98200: ['Bread - Roll, Soft White Round',
  'Veal - Osso Bucco',
  'Wine - White, Mosel Gold',
  'Wine - Magnotta - Belpaese',
  'Coffee - Irish Cream']}

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [78]:
recomendations_df = pd.DataFrame(recomendations)
recomendations_df

,98200
0,"Bread - Roll, Soft White Round"
1,Veal - Osso Bucco
2,"Wine - White, Mosel Gold"
3,Wine - Magnotta - Belpaese
4,Coffee - Irish Cream


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [79]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [60]:
new_customer_df = pd.DataFrame.from_dict(new_customer,orient='index')
new_customer_df = new_customer_df.rename(columns=({0:"99999"}))
new_customer_df

,99999
Cookies - Assorted,3
Flavouring - Orange,3
Fenngreek Seed,1
Wine - White Cab Sauv.on,1
Bandage - Flexible Neon,3
Oil - Shortening - All - Purpose,2
Beef - Montreal Smoked Brisket,4
French Pastry - Mini Chocolate,4
Snapple Lemon Tea,5
"Pepper - White, Ground",2


In [80]:
prod_cust_pivot[99999] = pd.DataFrame.from_dict(new_customer, orient="index")

In [61]:

prod_cust_pivot = prod_cust_pivot.fillna(0)
cust_prod_pivot = prod_cust_pivot.T
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)
similar_cust = list(cust_dist[99999].sort_values(ascending=False)[1:].head().index)
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[99999]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
1,Soup - Campbells Tomato Ravioli,63,0.0
2,Jagermeister,57,0.0
3,Bouq All Italian - Primerba,49,0.0
5,Foam Cup 6 Oz,44,0.0
6,Milk - 2%,41,0.0


### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [83]:
recomendations= {}

key_list = []
purchase_prodcuts = cust_prod_pivot.loc[99999][cust_prod_pivot.loc[99999] > 0]
items_purchased = [index for index,value in purchase_prodcuts.items()]

for item in items_purchased:
    top5 = cust_dist[item].sort_values(ascending=False)[1:].nlargest(5)
    key_list.append([keys for keys,value in top5.items()])

    flat_list = [item for sublist in key_list for item in sublist]
    filter_list = [item for item in flat_list if item not in items_purchased]

    countera = dict(Counter(filter_list))
    countera
    answer = [(key,countera[key]) for key in countera.keys()]
    answer
    answer_sorted = sorted(answer, key=lambda x: x[1],reverse=True)
    answer_sorted = [tupl[0] for tupl in answer_sorted]
    answer_sorted = answer_sorted[:5]
    recomendations[99999] = list(answer_sorted)
recomendations
    

{99999: ['Bread - Roll, Soft White Round',
  'Beans - Wax',
  'Veal - Osso Bucco',
  'Wine - White, Mosel Gold',
  'V8 - Berry Blend']}